In [1]:
import sys
sys.path.insert(0, '/home/tschernn/.local/opencv-cuda/lib/python3.10/dist-packages')

In [2]:
from keras.preprocessing.image import load_img
from skimage.feature import hog
from skimage import data, exposure
from scipy.spatial import distance
from tqdm import tqdm

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

2025-05-22 09:11:31.623613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-22 09:11:31.623663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-22 09:11:31.625037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 09:11:31.631842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-22 09:11:32.370488: W tensorflow/compiler/tf2

In [3]:
# make sure manually-compiled version of cv2 is used
import cv2
print(cv2.__file__)

/home/tschernn/.local/opencv-cuda/lib/python3.10/dist-packages/cv2/__init__.py


In [4]:
# check if cuda is available
if cv2.cuda.getCudaEnabledDeviceCount() == 1:
    print("CUDA detected!")

CUDA detected!


In [5]:
# load list of charter img paths

folders = [
    '../images/preprocessed_imgs/binary/writable_area/fr_notary',
    '../images/preprocessed_imgs/binary/writable_area/fr_papal',
    '../images/preprocessed_imgs/binary/writable_area/fr_other',
    '../images/preprocessed_imgs/binary/writable_area/at_notary',
    '../images/preprocessed_imgs/binary/writable_area/at_papal',
    '../images/preprocessed_imgs/binary/writable_area/at_other'
]
if isinstance(folders, str):
    folders = [folders]
charter_list = []
for folder in folders:
    files = os.listdir(folder)
    full_paths = [os.path.abspath(os.path.join(folder, f)) for f in files]
    charter_list.extend(full_paths)

charter_list

['/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_62f113115f5d2e4ead5b5c9e256bbe1e2.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_c7e12db9d6a8c09138e67b77818b1128.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_1a3f5a81f9bb7fabfe3012b2eccf4b17.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_667364aabe89e6dc65e939b80b212fa3.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_899fd4f8624fbaf5c0e426def2c0f493.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_abecb7d73e17bb5f70cb7c9052cad533.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_da478d335f3d171dab933ef1b82550f8.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_62f113115f5d2e4ead5b5c9e256bbe1e.jpg',
 '/home/tschernn/images/preprocessed_imgs/binary/writable_area/fr_notary/wa_102

In [6]:
charter_path = 'pickles/charter_list.pkl'

### export data as pickle file

with open(charter_path,'wb') as file:
    pickle.dump(charter_list, file)
    print(f'Saved features as {file.name}.')

Saved features as pickles/charter_list.pkl.


In [7]:
## set up HOG parameters

target_size = (1024, 1024)  # or (2048, 2048) or anything meaningful
cell_size = (64,64)
block_size = (128,128)
block_stride = (64,64)
nbins = 9

In [8]:
## extract features

all_features = []
image_names = []

for charter in charter_list:
    image = cv2.imread(charter)

    if image is None:
        print(f"Could not load {charter}")
        continue
    
    # grayscale & resize
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.equalizeHist(image_gray) # histogram equalization to deal with uneven illumination - optional
    image_resized = cv2.resize(image_gray, target_size)
    
    # initialize HOG descriptor for target_size
    hog = cv2.HOGDescriptor(target_size, block_size, block_stride, cell_size, nbins)
    
    # compute feature vector
    features = hog.compute(image_resized).flatten()

    all_features.append(features)
    image_names.append(charter)

In [9]:
## save results

# convert to NumPy array (will only work if all feature vectors have the same size!)
all_features = np.vstack(all_features)

with open(f'pickles/features_hog.pkl', 'wb') as handle:
    pickle.dump(all_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved {len(all_features)} feature vectors.")

Saved 312 feature vectors.


In [10]:
np.shape(all_features)

(312, 8100)